# In de cel na de functies kan je de functie visualize_first_and_second_last() 'callen'. 
# De 1ste en voorlaatste stand/tijdstip worden weergegeven. De laatste wordt genegeerd omdat deze 'meestal' experimenteel is. 
# Helaas is dit niet altijd het geval. Hier moet nog aan gewerkt worden om dat te kunnen onderscheiden...
# Fout in weergave van nieuwe task: als er onderweg ergens een task wordt gedelete en daarna terug toegevoegd wordt dit aangegeven als nieuwe task 
Onderaan een klad-versie van een plot van begin tot einde. Een bijverschijnsel is het afdrukken van een ongevraagde GIF file.

In [ ]:
#1.
#13/01/2026

# Static "Begin vs End" plot (NO animation, NO gif)
# - Plots TWO charts:
#     (1) first request of the day (earliest time)
#     (2) second-last request of the day (second-latest time)  <-- ignores the "real" last one (redundant)
#
# Shows:
#   - all request task coordinates (scatter)
#   - response route order (polyline) if matching response TXT exists
#   - route length (lon/lat units) in the title block
#
# Works with the standard naming convention:
#   ROUTE-YYYYMMDD-HHMMSS-NTASKS-NFIXED.json / .txt

# colors and legend adjustments

from pathlib import Path
import re
import json
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D

from time import time
start1 = time()

# ------------------------------------------------------------
# Styling knobs (edit these to change colors/markers/line styles)
# ------------------------------------------------------------
BASE_DOT_COLOR   = "blue"   # base task dots
NEW_DOT_COLOR    = "orange"     # new nodes (END only)
DEL_X_COLOR      = "red"        # deleted nodes (END only)

BASE_ROUTE_COLOR = "tab:blue"   # route polyline
NEW_EDGE_COLOR   = "green"      # new connections overlay

BASE_DOT_SIZE    = 12
NEW_DOT_SIZE     = 150
DEL_X_SIZE       = 80

BASE_ROUTE_LW    = 1.0
BASE_ROUTE_ALPHA = 0.60
NEW_EDGE_LW      = 2.0

# ------------------------------------------------------------
# Helpers (simple + readable)
# ------------------------------------------------------------
# File Name Regular Expression FN_RE
FN_RE = re.compile(
    r"^(?P<route>[^-]+)-(?P<date>\d{8})-(?P<time>\d{6})-(?P<n_tasks>\d+)-(?P<n_fixed>\d+)\.(?P<ext>json|txt)$",
    re.IGNORECASE
)

EARTH_RADIUS_KM = 6371.0088

def safe_json_load(p: Path) -> dict:
    return json.loads(p.read_text(encoding="utf-8", errors="ignore"))

def parse_time_from_filename(name: str) -> str | None:
    # 0521_301-20220614-053247-170-0.json -> "053247"
    parts = name.split("-")
    return parts[2] if len(parts) >= 3 else None

def node_key(lat: float, lon: float, nd: int = 6) -> str:
    return f"{round(float(lat), nd)}|{round(float(lon), nd)}"

def list_day_files(data_dir: str, subdir: str, route_prefix: str, date_str: str, ext: str) -> list[Path]:
    """
    Supports both layouts:
      A) data/requests/*.json
      B) data/requests/0521_301-20220614/*.json   <-- your case
    """
    base = Path(data_dir)
    root = base / subdir
    day_folder = root / f"{route_prefix}-{date_str}"

    if day_folder.exists():
        candidates = list(day_folder.glob(f"*.{ext}"))
    else:
        candidates = list(root.rglob(f"*.{ext}"))

    prefix = f"{route_prefix}-{date_str}-"
    files = [p for p in candidates if p.name.startswith(prefix)]
    files.sort(key=lambda p: parse_time_from_filename(p.name) or "999999")
    return files

def parse_request_tasks(request_file: Path) -> dict[str, tuple[float, float]]:
    """
    Returns: dict task_id -> (lat, lon)
    """
    obj = safe_json_load(request_file)
    tasks = {}
    for t in obj.get("tasks", []) or []:
        tid = str(t.get("id", "")).strip()
        addr = t.get("address", {}) or {}
        lat = float(addr.get("latitude"))
        lon = float(addr.get("longitude"))
        tasks[tid] = (lat, lon)
    return tasks

def read_response_tokens(response_file: Path) -> list[str]:
    """
    Response file = list of numbers (task ids) -> list[str]
    """
    return [
        ln.strip() for ln in response_file.read_text(encoding="utf-8", errors="ignore").splitlines()
        if ln.strip()
    ]

def map_response_to_coords(tasks: dict[str, tuple[float, float]], route_ids: list[str]):
    """
    Map response task-id sequence -> coords_latlon list, and count missing mappings.
    """
    coords = []
    missing = 0
    for rid in route_ids:
        if rid in tasks:
            coords.append(tasks[rid])
        else:
            missing += 1
    return coords, missing

def route_length_km(coords_latlon: list[tuple[float, float]]) -> float:
    """
    Great-circle (haversine) distance in kilometers along the route polyline.
    """
    if len(coords_latlon) < 2:
        return 0.0

    lat_deg = np.array([c[0] for c in coords_latlon], float)
    lon_deg = np.array([c[1] for c in coords_latlon], float)

    lat = np.radians(lat_deg)
    lon = np.radians(lon_deg)

    dlat = np.diff(lat)
    dlon = np.diff(lon)

    a = np.sin(dlat / 2.0) ** 2 + np.cos(lat[:-1]) * np.cos(lat[1:]) * np.sin(dlon / 2.0) ** 2
    c = 2.0 * np.arctan2(np.sqrt(a), np.sqrt(1.0 - a))

    return float(EARTH_RADIUS_KM * np.sum(c))

def pretty_title(p: Path) -> str:
    # 0521_301-20220614-053247-170-0.json
    parts = p.name.split("-")
    route = parts[0]
    date = parts[1]
    t = parts[2]
    return f"{route}  {date[:4]}-{date[4:6]}-{date[6:8]}  {t[:2]}:{t[2:4]}:{t[4:6]}"

def key_map_from_tasks(tasks: dict[str, tuple[float, float]], nd: int = 6) -> dict[str, tuple[float, float]]:
    """
    Returns: node_key -> (lat, lon)
    If multiple tasks share same coords, key collapses to one (OK for new/deleted node detection).
    """
    km = {}
    for (lat, lon) in tasks.values():
        km[node_key(lat, lon, nd=nd)] = (lat, lon)
    return km

def route_segments_from_coords(coords_latlon: list[tuple[float, float]], nd: int = 6) -> set[tuple[str, str]]:
    """
    Returns set of directed segments (k_i -> k_{i+1}) based on node_key(lat,lon).
    """
    if len(coords_latlon) < 2:
        return set()
    keys = [node_key(lat, lon, nd=nd) for (lat, lon) in coords_latlon]
    segs = set()
    for a, b in zip(keys[:-1], keys[1:]):
        if a != b:
            segs.add((a, b))
    return segs

# ------------------------------------------------------------
# Main: 2 static charts (BEGIN + SECOND_LAST)
# ------------------------------------------------------------
def visualize_first_and_second_last(data_dir: str, route_prefix: str, date_str: str):
    req_files = list_day_files(data_dir, "requests", route_prefix, date_str, "json")
    resp_files = list_day_files(data_dir, "responses", route_prefix, date_str, "txt")

    if len(req_files) < 2:
        raise FileNotFoundError("Need at least 2 request JSON files for this route/date.")

    resp_by_stem = {p.stem: p for p in resp_files}

    first_req = req_files[0]
    second_last_req = req_files[-2]   # ignore the "real" last one (redundant)

    def load_session(req_path: Path):
        tasks = parse_request_tasks(req_path)
        resp_path = resp_by_stem.get(req_path.stem)

        route_ids = read_response_tokens(resp_path) if resp_path else []
        coords_latlon, missing = map_response_to_coords(tasks, route_ids)

        all_lat = np.array([v[0] for v in tasks.values()], float)
        all_lon = np.array([v[1] for v in tasks.values()], float)
        route_lat = np.array([c[0] for c in coords_latlon], float)
        route_lon = np.array([c[1] for c in coords_latlon], float)

        return {
            "req": req_path,
            "resp": resp_path,
            "tasks": tasks,
            "all_lat": all_lat,
            "all_lon": all_lon,
            "route_latlon": coords_latlon,
            "route_lat": route_lat,
            "route_lon": route_lon,
            "missing": int(missing),
            "route_km": float(route_length_km(coords_latlon)),
        }

    s_begin = load_session(first_req)
    s_end = load_session(second_last_req)

    # --- new/deleted nodes (compare by rounded coordinate key)
    km_begin = key_map_from_tasks(s_begin["tasks"], nd=6)
    km_end = key_map_from_tasks(s_end["tasks"], nd=6)

    begin_keys = set(km_begin.keys())
    end_keys = set(km_end.keys())

    new_keys = end_keys - begin_keys
    deleted_keys = begin_keys - end_keys

    new_nodes_lat = np.array([km_end[k][0] for k in new_keys], float) if new_keys else np.array([], float)
    new_nodes_lon = np.array([km_end[k][1] for k in new_keys], float) if new_keys else np.array([], float)

    del_nodes_lat = np.array([km_begin[k][0] for k in deleted_keys], float) if deleted_keys else np.array([], float)
    del_nodes_lon = np.array([km_begin[k][1] for k in deleted_keys], float) if deleted_keys else np.array([], float)

    # --- new connections (segments in END route but not in BEGIN route)
    seg_begin = route_segments_from_coords(s_begin["route_latlon"], nd=6)
    seg_end = route_segments_from_coords(s_end["route_latlon"], nd=6)
    new_segs = seg_end - seg_begin

    # --- stable limits across both plots
    all_lon = np.concatenate([s_begin["all_lon"], s_end["all_lon"]]) if len(s_begin["all_lon"]) and len(s_end["all_lon"]) else (s_begin["all_lon"] if len(s_begin["all_lon"]) else s_end["all_lon"])
    all_lat = np.concatenate([s_begin["all_lat"], s_end["all_lat"]]) if len(s_begin["all_lat"]) and len(s_end["all_lat"]) else (s_begin["all_lat"] if len(s_begin["all_lat"]) else s_end["all_lat"])

    x_min, x_max = float(np.min(all_lon)), float(np.max(all_lon))
    y_min, y_max = float(np.min(all_lat)), float(np.max(all_lat))
    pad_x = (x_max - x_min) * 0.05 if x_max > x_min else 0.01
    pad_y = (y_max - y_min) * 0.05 if y_max > y_min else 0.01

    fig = plt.figure(figsize=(18, 7))

    # -------------------- Plot 1: BEGIN --------------------
    ax0 = plt.subplot(1, 2, 1)
    ax0.set_title(f"BEGIN (first request)\n{pretty_title(s_begin['req'])}")
    ax0.set_xlabel("Longitude")
    ax0.set_ylabel("Latitude")
    ax0.set_xlim(x_min - pad_x, x_max + pad_x)
    ax0.set_ylim(y_min - pad_y, y_max + pad_y)
    ax0.grid(True)

    ax0.scatter(s_begin["all_lon"], s_begin["all_lat"], s=BASE_DOT_SIZE, color=BASE_DOT_COLOR)
    if len(s_begin["route_lon"]) >= 2:
        ax0.plot(s_begin["route_lon"], s_begin["route_lat"], linewidth=BASE_ROUTE_LW, color=BASE_ROUTE_COLOR)

    ax0.text(
        0.01, 0.99,
        f"tasks: {len(s_begin['all_lon'])}   route_nodes: {len(s_begin['route_lon'])}\n"
        f"missing_mapped: {s_begin['missing']}   route: {s_begin['route_km']:.3f}km",
        transform=ax0.transAxes, va="top"
    )

    # -------------------- Plot 2: END (second_last) --------------------
    ax1 = plt.subplot(1, 2, 2)
    ax1.set_title(f"END (second_last request)\n{pretty_title(s_end['req'])}")
    ax1.set_xlabel("Longitude")
    ax1.set_ylabel("Latitude")
    ax1.set_xlim(x_min - pad_x, x_max + pad_x)
    ax1.set_ylim(y_min - pad_y, y_max + pad_y)
    ax1.grid(True)

    # base nodes
    ax1.scatter(s_end["all_lon"], s_end["all_lat"], s=BASE_DOT_SIZE, color=BASE_DOT_COLOR)

    # new nodes
    if len(new_nodes_lon):
        ax1.scatter(new_nodes_lon, new_nodes_lat, s=NEW_DOT_SIZE, color=NEW_DOT_COLOR)

    # deleted nodes (X)
    if len(del_nodes_lon):
        ax1.scatter(del_nodes_lon, del_nodes_lat, s=DEL_X_SIZE, marker="x", color=DEL_X_COLOR)

    # base route line
    if len(s_end["route_lon"]) >= 2:
        ax1.plot(
            s_end["route_lon"], s_end["route_lat"],
            linewidth=BASE_ROUTE_LW, alpha=BASE_ROUTE_ALPHA, color=BASE_ROUTE_COLOR
        )

    # overlay NEW connections (green, thicker)
    for a, b in new_segs:
        if a in km_end and b in km_end:
            lat_a, lon_a = km_end[a]
            lat_b, lon_b = km_end[b]
        elif a in km_begin and b in km_begin:
            lat_a, lon_a = km_begin[a]
            lat_b, lon_b = km_begin[b]
        else:
            continue
        ax1.plot([lon_a, lon_b], [lat_a, lat_b], color=NEW_EDGE_COLOR, linewidth=NEW_EDGE_LW)

    # Legend (top right) using proxy artists so it stays clean
    legend_handles = [
        Line2D([], [], linestyle="none", marker="o", color=BASE_DOT_COLOR, label="Location"),
        Line2D([], [], linestyle="none", marker="o", color=NEW_DOT_COLOR,  label="New Location"),
        Line2D([], [], linestyle="none", marker="x", color=DEL_X_COLOR,    label="Deleted Location"),
        Line2D([], [], linestyle="-",    color=BASE_ROUTE_COLOR, linewidth=BASE_ROUTE_LW, label="Route"),
        Line2D([], [], linestyle="-",    color=NEW_EDGE_COLOR,   linewidth=NEW_EDGE_LW,  label="New Route"),
    ]
    ax1.legend(handles=legend_handles, loc="upper right", framealpha=0.9)

    ax1.text(
        0.01, 0.99,
        f"tasks: {len(s_end['all_lon'])}   route_nodes: {len(s_end['route_lon'])}\n"
        f"missing_mapped: {s_end['missing']}   route: {s_end['route_km']:.3f}km \n"
        f"new_nodes: {len(new_keys)}   deleted_nodes: {len(deleted_keys)}   new_edges: {len(new_segs)}",
        transform=ax1.transAxes, va="top"
    )

    print(f"Found request files: {len(req_files)}, response files: {len(resp_files)}")
    print(f"begin request: {s_begin['req'].name}                               end request: {s_end['req'].name}")
    print(f"begin response: {s_begin['resp'].name if s_begin['resp'] else 'MISSING'}                               end response: {s_end['resp'].name}")

    plt.tight_layout()
    plt.show()

print(f"Duration:{time() - start1:.3f}")


In [ ]:
#2.

# ------------------------------------------------------------
# CALL
# ------------------------------------------------------------
start1 = time()

visualize_first_and_second_last(
    data_dir=r"C:\Users\dirk1\0.SYNTRA1\syntra_ds1\Playground1\opdracht_transport\learning_driver_preferences\data",
    route_prefix="0521_301",
    date_str="20220614",
)

print(f"Duration:{time() - start1:.3f}")
#1.4s

In [ ]:
# all plots
# RUN ONCE — Minimal, readable "Trim/Add + Route Order" visualizer
# - Uses request JSON + response TXT files with the naming pattern:
#     ROUTE-YYYYMMDD-HHMMSS-NTASKS-NFIXED.json / .txt
# - Shows: current nodes, added nodes, removed nodes, current route polyline + faint previous polyline
#
# External requirement:
#   - safe_json_load(p: Path) must exist (you already have it in your big cell)

from pathlib import Path
import re, json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, PillowWriter
from IPython.display import HTML, display

from time import time
start1 = time()

# -----------------------------
# 1) Filename parsing
# -----------------------------
FN_RE = re.compile(
    r"^(?P<route>[^-]+)-(?P<date>\d{8})-(?P<time>\d{6})-(?P<n_tasks>\d+)-(?P<n_fixed>\d+)\.(?P<ext>json|txt)$",
    re.IGNORECASE
)

def safe_json_load(p: Path) -> dict:
    return json.loads(p.read_text(encoding="utf-8", errors="ignore"))

def parse_meta(filename: str) -> dict | None:
    m = FN_RE.match(filename)
    if not m:
        return None
    return {
        "route": m.group("route"),
        "date": m.group("date"),
        "time": m.group("time"),
        "n_tasks_in_name": int(m.group("n_tasks")),
        "n_fixed_in_name": int(m.group("n_fixed")),
        "ext": m.group("ext").lower(),
    }

# -----------------------------
# 2) File discovery (recursive)
# -----------------------------
def find_files(root: Path, ext: str, route: str, yyyymmdd: str) -> list[Path]:
    hits = []
    for p in root.rglob(f"*.{ext}"):
        meta = parse_meta(p.name)
        if meta and meta["route"] == route and meta["date"] == yyyymmdd:
            hits.append(p)
    hits.sort(key=lambda p: parse_meta(p.name)["time"])
    return hits

# -----------------------------
# 3) Data readers
# -----------------------------
def node_key(lat, lon, nd=6) -> str | None:
    if pd.isna(lat) or pd.isna(lon):
        return None
    return f"{round(float(lat), nd)}|{round(float(lon), nd)}"

def read_response_tokens(txt_path: Path) -> list[int]:
    txt = txt_path.read_text(encoding="utf-8", errors="ignore")
    return [int(t) for t in re.findall(r"\d+", txt)]

def parse_request_tasks(json_path: Path) -> pd.DataFrame:
    obj = safe_json_load(json_path)
    tasks = obj.get("tasks", []) or []
    rows = []
    for i, t in enumerate(tasks):
        addr = t.get("address", {}) or {}
        tw = t.get("timeWindow", {}) or {}
        lat = addr.get("latitude", np.nan)
        lon = addr.get("longitude", np.nan)
        rows.append({
            "task_idx": i,
            "task_id": str(t.get("id", "")).strip(),
            "latitude": float(lat) if lat is not None else np.nan,
            "longitude": float(lon) if lon is not None else np.nan,
            "tw_from": tw.get("from"),
            "tw_till": tw.get("till"),
        })
    df = pd.DataFrame(rows)
    df["node_key"] = [node_key(a, b) for a, b in zip(df["latitude"], df["longitude"])]
    return df.dropna(subset=["node_key"]).reset_index(drop=True)

# -----------------------------
# 4) Map response order -> coords (task_id vs task_idx)
# -----------------------------
def map_response_to_coords(resp_tokens: list[int], tasks_df: pd.DataFrame) -> tuple[np.ndarray, str, int]:
    if not resp_tokens:
        return np.empty((0, 2)), "missing", 0

    # try tokens as task_id
    toks_as_id = [str(t) for t in resp_tokens]
    id_join = pd.DataFrame({"task_id": toks_as_id}).merge(
        tasks_df[["task_id", "longitude", "latitude"]],
        on="task_id",
        how="left"
    )
    coords_id = id_join[["longitude", "latitude"]].to_numpy(float)
    miss_id = int(np.isnan(coords_id).any(axis=1).sum())

    # try tokens as task_idx
    idx_join = pd.DataFrame({"task_idx": resp_tokens}).merge(
        tasks_df[["task_idx", "longitude", "latitude"]],
        on="task_idx",
        how="left"
    )
    coords_idx = idx_join[["longitude", "latitude"]].to_numpy(float)
    miss_idx = int(np.isnan(coords_idx).any(axis=1).sum())

    if miss_id <= miss_idx:
        return coords_id, "task_id", miss_id
    return coords_idx, "index", miss_idx

# -----------------------------
# 5) Route length (polyline length in lon/lat units)
# -----------------------------
def route_length(coords: np.ndarray) -> float:
    if coords is None or len(coords) < 2:
        return 0.0
    d = np.diff(coords, axis=0)
    seg = np.sqrt(d[:, 0] ** 2 + d[:, 1] ** 2)
    return float(np.nansum(seg))

# -----------------------------
# 6) Main visualizer
# -----------------------------
def run_visualization(
    *,
    data_dir: str,
    route_prefix: str,
    yyyymmdd: str,
    requests_subdir: str = "requests",
    responses_subdir: str = "responses",
    interval_ms: int = 1200,
    show_table: bool = False,
    #save_gif: bool = False,
    #gif_name: str | None = None,
):
    data_dir = Path(data_dir)
    req_root = data_dir / requests_subdir
    resp_root = data_dir / responses_subdir

    req_files = find_files(req_root, "json", route_prefix, yyyymmdd)
    resp_files = find_files(resp_root, "txt", route_prefix, yyyymmdd)
    if not req_files:
        raise FileNotFoundError(f"No request JSON files for {route_prefix} on {yyyymmdd} under {req_root}")
    if not resp_files:
        raise FileNotFoundError(f"No response TXT files for {route_prefix} on {yyyymmdd} under {resp_root}")

    resp_by_stem = {p.stem: p for p in resp_files}

    # ---- build sessions list
    sessions = []
    for req in req_files:
        meta = parse_meta(req.name)
        tasks_df = parse_request_tasks(req)

        resp = resp_by_stem.get(req.stem)
        tokens = read_response_tokens(resp) if resp else []
        coords, resp_type, miss = map_response_to_coords(tokens, tasks_df)

        sessions.append({
            "meta": {
                "date": meta["date"],
                "time": meta["time"],
                "file": req.name,
                "stem": req.stem,
                "n_fixed": meta["n_fixed_in_name"],
                "n_tasks": int(len(tasks_df)),
                "resp_type": resp_type,
                "miss": int(miss),
                "route_len": route_length(coords),
            },
            "tasks_df": tasks_df,
            "coords": coords,
        })

    if show_table:
        display(pd.DataFrame([s["meta"] for s in sessions]))

    # ---- precompute per-frame arrays (added/removed + current + route lines)
    added_xy, removed_xy, current_xy = [], [], []
    route_xy, prev_route_xy = [], []
    labels, cur_len, prev_len = [], [], []

    for i, s in enumerate(sessions):
        df = s["tasks_df"]
        cur_keys = set(df["node_key"])

        if i == 0:
            prev_df = df.iloc[0:0]
            prev_keys = set()
            prev_coords = np.empty((0, 2))
        else:
            prev_df = sessions[i - 1]["tasks_df"]
            prev_keys = set(prev_df["node_key"])
            prev_coords = sessions[i - 1]["coords"]

        add_keys = cur_keys - prev_keys
        rem_keys = prev_keys - cur_keys

        added_xy.append(df[df["node_key"].isin(add_keys)][["longitude", "latitude"]].to_numpy(float))
        removed_xy.append(prev_df[prev_df["node_key"].isin(rem_keys)][["longitude", "latitude"]].to_numpy(float))
        current_xy.append(df[["longitude", "latitude"]].to_numpy(float))

        route_xy.append(s["coords"])
        prev_route_xy.append(prev_coords)

        cur_len.append(route_length(s["coords"]))
        prev_len.append(route_length(prev_coords))

        t = s["meta"]["time"]
        labels.append(
            f"{s['meta']['date']} {t[:2]}:{t[2:4]}:{t[4:6]}  "
            f"n={s['meta']['n_tasks']} fixed={s['meta']['n_fixed']} resp={s['meta']['resp_type']} miss={s['meta']['miss']}"
        )

    # stable axis limits
    all_pts = np.vstack([xy for xy in current_xy if len(xy)])
    x_min, y_min = np.nanmin(all_pts, axis=0)
    x_max, y_max = np.nanmax(all_pts, axis=0)
    pad_x = (x_max - x_min) * 0.05 if x_max > x_min else 0.01
    pad_y = (y_max - y_min) * 0.05 if y_max > y_min else 0.01

    # ---- plot elements
    fig = plt.figure()
    ax = plt.gca()
    ax.set_title("Trim/Add + Route Order (current + previous)")
    ax.set_xlabel("Longitude")
    ax.set_ylabel("Latitude")
    ax.set_xlim(x_min - pad_x, x_max + pad_x)
    ax.set_ylim(y_min - pad_y, y_max + pad_y)
    ax.grid(True)

    sc_cur = ax.scatter([], [], s=10, marker="o")
    sc_add = ax.scatter([], [], s=30, marker="o")
    sc_rem = ax.scatter([], [], s=60, marker="x")

    ln_prev, = ax.plot([], [], linewidth=1, alpha=0.10)
    ln_route, = ax.plot([], [], linewidth=1, alpha=1.00)

    txt = ax.text(0.01, 0.99, "", transform=ax.transAxes, va="top")

    def init():
        sc_cur.set_offsets(np.empty((0, 2)))
        sc_add.set_offsets(np.empty((0, 2)))
        sc_rem.set_offsets(np.empty((0, 2)))
        ln_prev.set_data([], [])
        ln_route.set_data([], [])
        txt.set_text("")
        return sc_cur, sc_add, sc_rem, ln_prev, ln_route, txt

    def update(i):
        sc_cur.set_offsets(current_xy[i] if len(current_xy[i]) else np.empty((0, 2)))
        sc_add.set_offsets(added_xy[i] if len(added_xy[i]) else np.empty((0, 2)))
        sc_rem.set_offsets(removed_xy[i] if len(removed_xy[i]) else np.empty((0, 2)))

        if len(prev_route_xy[i]):
            ln_prev.set_data(prev_route_xy[i][:, 0], prev_route_xy[i][:, 1])
        else:
            ln_prev.set_data([], [])

        if len(route_xy[i]):
            ln_route.set_data(route_xy[i][:, 0], route_xy[i][:, 1])
        else:
            ln_route.set_data([], [])

        txt.set_text(
            f"{labels[i]}\n"
            f"added: {len(added_xy[i])} removed: {len(removed_xy[i])}\n"
            f"len(prev): {prev_len[i]:.6f}    len(cur): {cur_len[i]:.6f}"
        )
        return sc_cur, sc_add, sc_rem, ln_prev, ln_route, txt

    anim = FuncAnimation(fig, update, frames=len(sessions), init_func=init, interval=interval_ms, blit=True)
    display(HTML(anim.to_jshtml()))

#    if save_gif:
#        if gif_name is None:
#            gif_name = f"trim_add_{route_prefix}_{yyyymmdd}.gif"
#        out_gif = data_dir / gif_name
#        anim.save(out_gif, writer=PillowWriter(fps=max(1, int(1000 / interval_ms))))
#        print("Saved GIF:", out_gif)

    return anim


print(f"Duration:{time() - start1:.3f}")



In [ ]:
from time import time
start1 = time()

_ = run_visualization(
    data_dir=r"C:\Users\dirk1\0.SYNTRA1\syntra_ds1\Playground1\opdracht_transport\learning_driver_preferences\data",
    route_prefix="0521_301",
    yyyymmdd="20220614",
    #save_gif=False,
)

print(f"Duration:{time() - start1:.3f}")
